<a href="https://colab.research.google.com/github/Navak94/deeplearning_HW4/blob/main/cnn_improved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

If you decide to participate in the competition, save the **predictions.json and flops_and_params.json** files in a zip folder with your group name.

In [1]:
# import packages
import pandas as pd
import numpy as np
import torch
import json
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [2]:
# connect to google drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# Step 1

In [3]:
# NEW TOM + NATE BLOCK
import torch.nn as nn

class betterCNN(nn.Module):
    def __init__(self, num_classes=5):
        super(betterCNN, self).__init__()

        # convolutional block
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)

        # convolutional block
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(128)

        # convolutional block
        self.conv5 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(256)
        self.conv6 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
        self.bn6 = nn.BatchNorm2d(256)

        # pooling and dropout
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.2)

        # fully connected layers
        self.fc1 = nn.Linear(256 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        # 1 block
        x = self.conv1(x)
        x = self.bn1(x)
        x = torch.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = torch.relu(x)
        x = self.pool(x)

        # 2 block
        x = self.conv3(x)
        x = self.bn3(x)
        x = torch.relu(x)
        x = self.conv4(x)
        x = self.bn4(x)
        x = torch.relu(x)
        x = self.pool(x)

        # 3 block
        x = self.conv5(x)
        x = self.bn5(x)
        x = torch.relu(x)
        x = self.conv6(x)
        x = self.bn6(x)
        x = torch.relu(x)
        x = self.pool(x)

        # flatten
        x = x.view(-1, 256 * 8 * 8)

        # fully connected layers
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)

        return x

model = betterCNN(num_classes=5)
model = model.to(device)
print(model)

betterCNN(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn5): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv6): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_run

# Step 2

In [4]:
# NEW TOM + NATE BLOCK

from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# transform with augmentation
train_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# val transform (no augmentation)
val_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset_dir = '/content/drive/MyDrive/train_set'

full_dataset = datasets.ImageFolder(root=trainset_dir, transform=None)

total_size = len(full_dataset)
indices = list(range(total_size))
np.random.seed(42)
np.random.shuffle(indices)

train_size = int(0.85 * total_size)
val_size = total_size - train_size

train_indices = indices[0:train_size]
val_indices = indices[train_size:total_size]

train_dataset = datasets.ImageFolder(root=trainset_dir, transform=train_transform)
train_subset = torch.utils.data.Subset(train_dataset, train_indices)

val_dataset = datasets.ImageFolder(root=trainset_dir, transform=val_transform)
val_subset = torch.utils.data.Subset(val_dataset, val_indices)

train_loader = DataLoader(train_subset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=64, shuffle=False)


# Step 3

In [5]:
# NEW TOM + NATE BLOCK

import torch.optim as optim
criterion = nn.CrossEntropyLoss(label_smoothing=0.09) # added label smoothing
optimizer = optim.Adam(model.parameters(), lr=0.002, weight_decay=0.00001) # added weight decay
# added learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='max',
    factor=0.5,
    patience=5)

# Step 4

In [6]:
# NEW TOM + NATE BLOCK

def train(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=30):
    best_val_acc = 0.0
    patience_counter = 0
    early_stop_patience = 10

    train_losses = []
    val_accuracies = []

    for epoch in range(num_epochs):
        # train
        model.train()
        running_loss = 0.0
        train_correct = 0
        train_total = 0

        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss = running_loss + loss.item()

            predictions = torch.max(outputs, 1)[1]
            train_total = train_total + labels.size(0)
            train_correct = train_correct + (predictions == labels).sum().item()

        avg_train_loss = running_loss / len(train_loader)
        train_acc = 100.0 * train_correct / train_total
        train_losses.append(avg_train_loss)

        # val
        model.eval()
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = model(inputs)
                predictions = torch.max(outputs, 1)[1]
                val_total = val_total + labels.size(0)
                val_correct = val_correct + (predictions == labels).sum().item()

        val_acc = 100.0 * val_correct / val_total
        val_accuracies.append(val_acc)

        print(f"Epoch {epoch + 1}/{num_epochs}")
        print(f" -> Train Loss: {avg_train_loss:.4f}, Train Acc: {train_acc:.2f}%")
        print(f" -> Val Acc: {val_acc:.2f}%")

        # new scheduler step
        scheduler.step(val_acc)

        # save state added
        if val_acc >= best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_model.pth')
            print(f"----> Best model saved ---> Val Acc: {val_acc:.2f}%")
            patience_counter = 0
        else:
            patience_counter = patience_counter + 1

        # early stopping
        if patience_counter >= early_stop_patience:
            break

        print()

    return train_losses, val_accuracies

train_losses, val_accuracies = train(
    model,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    scheduler,
    num_epochs=100
)


Epoch 1/100
 -> Train Loss: 5.0669, Train Acc: 28.71%
 -> Val Acc: 37.81%
----> Best model saved ---> Val Acc: 37.81%

Epoch 2/100
 -> Train Loss: 1.5103, Train Acc: 34.02%
 -> Val Acc: 40.26%
----> Best model saved ---> Val Acc: 40.26%

Epoch 3/100
 -> Train Loss: 1.5076, Train Acc: 34.93%
 -> Val Acc: 43.10%
----> Best model saved ---> Val Acc: 43.10%

Epoch 4/100
 -> Train Loss: 1.5053, Train Acc: 35.46%
 -> Val Acc: 43.29%
----> Best model saved ---> Val Acc: 43.29%

Epoch 5/100
 -> Train Loss: 1.4824, Train Acc: 37.87%
 -> Val Acc: 43.86%
----> Best model saved ---> Val Acc: 43.86%

Epoch 6/100
 -> Train Loss: 1.4767, Train Acc: 37.50%
 -> Val Acc: 47.26%
----> Best model saved ---> Val Acc: 47.26%

Epoch 7/100
 -> Train Loss: 1.4526, Train Acc: 39.81%
 -> Val Acc: 49.53%
----> Best model saved ---> Val Acc: 49.53%

Epoch 8/100
 -> Train Loss: 1.4357, Train Acc: 41.58%
 -> Val Acc: 45.94%

Epoch 9/100
 -> Train Loss: 1.4142, Train Acc: 43.52%
 -> Val Acc: 45.75%

Epoch 10/100
 -> 

# Step 5

In [ ]:
#| label: prob1
# NEW TOM + NATE BLOCK

def evaluate(model, val_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            predictions = torch.max(outputs, 1)[1]
            total = total + labels.size(0)
            correct = correct + (predictions == labels).sum().item()

    accuracy = 100.0 * correct / total
    print(f'Val Accuracy: {accuracy:.2f}%')
    return accuracy

model.load_state_dict(torch.load('best_model.pth'))
evaluate(model, val_loader)

Val Accuracy: 69.94%


69.94328922495274

Please don't make any change after this line. The only parameters you may modify are those within the "test_transform" function.

In [8]:
import os
from PIL import Image
class CustomImageDataset(torch.utils.data.Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.image_paths = [os.path.join(folder_path, filename) for filename in os.listdir(folder_path) if filename.endswith(('png', 'jpg', 'jpeg'))]

    def __len__(self):
        return len(self.image_paths)

    def filename2index(self, filename):
        return os.path.basename(filename).replace('.jpg', '')

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img, self.filename2index(img_path)

In [9]:
test_folder = '/content/drive/MyDrive/test_set'
test_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
test_dataset = CustomImageDataset(test_folder, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

class_to_idx = train_dataset.class_to_idx
idx_to_class = {v: k for k, v in class_to_idx.items()}

In [10]:
# Make predictions
def evaluate_model(model, test_loader, idx_to_class):
    all_predictions = {}
    with torch.no_grad():
        for inputs, index in test_loader:
            inputs = inputs.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            predicted_class = predicted.item()
            predicted_class_name = idx_to_class[predicted_class]
            all_predictions[index[0]] = predicted_class_name

    return all_predictions

predictions = evaluate_model(model, test_loader, idx_to_class)
with open('predictions.json', 'w') as json_file:
    json.dump(predictions, json_file, indent=4)

print("Evaluation completed and predictions saved.")

Evaluation completed and predictions saved.


In [11]:
# you may need to install thop when you first run this code
!pip install thop

In [12]:
# Compute FLOPs using thop
import thop
input_tensor = test_dataset[0][0].unsqueeze(0).to(device) # must have exact same size of the data input (batch, channel, height, width) and be on the same device as the model
flops, params = thop.profile(model, inputs=(input_tensor,))
print(f"FLOPs: {flops}")
print(f"Number of Parameters: {params}")
flops_and_params = {
    "FLOPs": flops,
    "Parameters": params
}

output_json_path = 'flops_and_params.json'

with open(output_json_path, 'w') as json_file:
    json.dump(flops_and_params, json_file, indent=4)

print(f"FLOPs and parameters have been saved to {output_json_path}")

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
FLOPs: 623118848.0
Number of Parameters: 9538885.0
FLOPs and parameters have been saved to flops_and_params.json
